<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeClassificacaoDaBaseDeDados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Base de Dados Twitter

Análise da base de dados do Twitter com linguagem ofensiva

Classificação

Import e instalação das bibliotecas necessárias

In [ ]:
#Install e downloads
!pip install emoji
!pip install pandas
!pip install nltk
#nltk.download('all-nltk') #Demora um pouco

     |████████████████████████████████| 175 kB 15.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=a08289a7ba3797018c6352a3bd5be23c477ecc497b0b4adac91a54208afec01b
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
#Imports
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import re
import itertools

In [ ]:
# Padrão RegEx
PADRAO_PALAVRAS_PT = r'[a-zA-Zà-úÀ-Ú]+'

########## PATHS DE ARQUIVOS ##########

# Base após pré-processamento COM stopwords (Base 1)
PATH_BASE1_COM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base1_Com_StopWords.csv"  

# Base após pré-processamento SEM stopwords (Base 2)
PATH_BASE2_SEM_STOPWORDS = r"/content/drive/MyDrive/TCC/dados/processadas/Base2_Sem_StopWords.csv"

# Amostra Base 1
PATH_TESTE_BASE1 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base1_Com_StopWords.csv"  

# Amostra Base 2
PATH_TESTE_BASE2 = r"/content/drive/MyDrive/TCC/dados/processadas/Teste_Base2_Sem_StopWords.csv"

# Lista de Hates
PATH_LIST_HATE_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/odio_list.csv"

# Lista de Palavrões
PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR = r"/content/drive/MyDrive/TCC/dados/dicionarios/badword_list.csv"

In [ ]:
#Carrega a base de dados com stopWords
base1 = pd.read_csv(PATH_BASE1_COM_STOPWORDS, index_col=0)
#base1 = pd.read_csv(PATH_TESTE_BASE1, index_col=0)
base1.head()

,text
1,retwet eu queria compartilhar mas antes eu gos...
2,gostei da pulseira e da educação dele nossaaa ...
3,esses não irão ao encontro e beijing
4,cadê eles está faltando
5,e eu me questionando quem é esse blogueiro sem...


In [ ]:
#Carrega a base de dados sem stopWords
base2= pd.read_csv(PATH_BASE2_SEM_STOPWORDS, index_col=0)
#base2= pd.read_csv(PATH_TESTE_BASE2, index_col=0)
base2.head()

,text
1,retwet queria compartilhar antes gostaria sabe...
2,gostei pulseira educação nossaaa muita elegânc...
3,irão encontro beijing
5,questionando é blogueiro blog juntos
6,renan caloteiro desonesto vejam link abaixo


In [ ]:
base1.shape

(73462, 1)

In [ ]:
base2.shape

(64847, 1)

Classificação Hate, Hate+ e outros

In [ ]:
# Carrega dicionário hate
dict_hate = pd.read_csv(PATH_LIST_HATE_EXPRESSIONS_PT_BR)
dict_hate.head(5)

,hatelist
0,aberração
1,abestado
2,abestalhado
3,aborto
4,acabouamamata


In [ ]:
# Carrega dicionário badword
dict_badword = pd.read_csv(PATH_LIST_BADWORDS_EXPRESSIONS_PT_BR)
dict_badword.head(5)

,badword_list
0,babaca
1,boceta
2,boquete
3,bosseta
4,buceta


In [ ]:
# Definição da função de classificação

def classifica(tweet, dicionario, LABEL1, LABEL2):
  # Tokeniza o tweet  
  tokenizer = RegexpTokenizer(PADRAO_PALAVRAS_PT)
  tokens = tokenizer.tokenize(tweet)
  
  chave = dicionario.keys()[0]

  lista = list(dicionario[chave])
  lista_expressoes = []

  # Gera lista de expressões
  for termo in lista:
    termo_tokens = tokenizer.tokenize(termo)
    if(len(termo_tokens)>1): #expressoes
      lista_expressoes.append(termo)

  # Verifica se o tweet possui ao menos uma PALAVRA de hate
  for token in tokens:
    if token in lista:
      return LABEL1

  # Verifica se o tweet possui ao menos uma EXPRESSÃO de hate
  for palavra in lista_expressoes:
    result = re.subn(r'^' + palavra + '\s', palavra + ' ', tweet) # Substitui no início
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '$',' '+ palavra, tweet) # Substitui no final
    if(result[1]):
      break
    result = re.subn(r'\s' + palavra + '\s', ' '+palavra+ ' ', tweet) #Substitui no meio
    if(result[1]):
      break

  if(result[1]):
    return LABEL1
  else:
    return LABEL2

In [ ]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 1
base1['label'] = base1['text'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base1.head()

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate


In [ ]:
# CLASSIFICAÇÃO HATE
# Executa a classificação na base 2
base2['label'] = base2['text'].apply(lambda x: classifica(x,dict_hate,"hate","not_hate"))
base2.head()

,text,label
1,retwet queria compartilhar antes gostaria sabe...,not_hate
2,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,irão encontro beijing,not_hate
5,questionando é blogueiro blog juntos,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate


In [ ]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base1['classificacao_badword'] = base1['text'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base1.head()

,text,label,classificacao_badword
1,retwet eu queria compartilhar mas antes eu gos...,not_hate,not_badword
2,gostei da pulseira e da educação dele nossaaa ...,not_hate,not_badword
3,esses não irão ao encontro e beijing,not_hate,not_badword
4,cadê eles está faltando,not_hate,not_badword
5,e eu me questionando quem é esse blogueiro sem...,not_hate,not_badword


In [ ]:
# CLASSIFICAÇÃO BADWORD
# Executa a classificação na base 1
base2['classificacao_badword'] = base2['text'].apply(lambda x: classifica(x,dict_badword,"badword","not_badword"))
base2.head()

,text,label,classificacao_badword
1,retwet queria compartilhar antes gostaria sabe...,not_hate,not_badword
2,gostei pulseira educação nossaaa muita elegânc...,not_hate,not_badword
3,irão encontro beijing,not_hate,not_badword
5,questionando é blogueiro blog juntos,not_hate,not_badword
6,renan caloteiro desonesto vejam link abaixo,not_hate,not_badword


In [ ]:
# BASE 1 COM STOP WORDS
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop1 = base1[ (base1['classificacao_badword']=='badword') & (base1['label']=='not_hate')].index 
base1.drop(baseDrop1, inplace=True)

# Dropa colunas que não serão utilizadas
base1.drop(['classificacao_badword'], axis='columns', inplace=True)

In [ ]:
base1

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
...,...,...
82244,abençoado ano novo sua linda e querida obrigad...,not_hate
82245,leva eles com a gente eles são amorzinhos d,not_hate
82246,k sai daqui marivaldo tu me odeia né pelo jeito,hate
82247,se tu nunca fumou maconha cole isso no seu mur...,not_hate


In [ ]:
# BASE 2 SEM STOP WORDS
# ELIMINAR OS TWEETS QUE CONTEM BADWORDS E SÃO NOT_HATE : drop (badword and not_hate)

baseDrop2= base2[ (base2['classificacao_badword']=='badword') & (base2['label']=='not_hate')].index 
base2.drop(baseDrop2, inplace=True)

# Dropa colunas que não serão utilizadas
base2.drop(['classificacao_badword'], axis='columns', inplace=True)

In [ ]:
base2

,text,label
1,retwet queria compartilhar antes gostaria sabe...,not_hate
2,gostei pulseira educação nossaaa muita elegânc...,not_hate
3,irão encontro beijing,not_hate
5,questionando é blogueiro blog juntos,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate
...,...,...
82242,egoísmo é grande parte produto sociedade émile...,not_hate
82244,abençoado novo linda querida obrigada carinho ...,not_hate
82245,leva gente amorzinhos d,not_hate
82246,k sai daqui marivaldo odeia né jeito,hate


# Definições balanceamento


In [ ]:
categorias_desejadas = ['hate', 'not_hate'] # !!!

def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

# Balanceia BASE 1

In [ ]:
# Agrupa dados por label
base1.groupby('label').count()

In [ ]:
df_filtrado1 = base1[base1.label.isin(categorias_desejadas)]
df_filtrado1.shape

In [ ]:
df_filtrado1

In [ ]:
# Aplica função de balancear os dados
dataBC1 = balanceia_dados(df_filtrado1)

# DATA BALANCEADO
dataBC1.reset_index(drop=True, inplace=True) # reseta index
dataBC1.shape

In [ ]:
# convert output  hate =1 not_hate =0
dataBC1.label = [1 if each == "hate" else 0 for each in dataBC1.label] 

In [ ]:
dataBC1.groupby('label').count()

In [ ]:
# Salva a base1 classificada e balanceada
base1.to_csv("/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada_balanceada.csv") 

# Balanceia BASE 2

In [ ]:
# Agrupa dados por label
base2.groupby('label').count()

In [ ]:
df_filtrado2 = base2[base2.label.isin(categorias_desejadas)]
df_filtrado2.shape

In [ ]:
df_filtrado2

In [ ]:
# Aplica função de balancear os dados
dataBC2 = balanceia_dados(df_filtrado2)

# DATA BALANCEADO
dataBC2.reset_index(drop=True, inplace=True) # reseta index
dataBC2.shape

In [ ]:
# convert output  hate =1 not_hate =0
dataBC2.label = [1 if each == "hate" else 0 for each in dataBC2.label] 

In [ ]:
dataBC2.groupby('label').count()

In [ ]:
# Salva a base2 classificada e balanceada
base2.to_csv("/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada_balanceada.csv")